In [47]:
import open3d as o3d
import numpy as np
from sklearn.neighbors import NearestNeighbors
import os

In [ ]:
datasetPath = "/PublicData/CadDataset/a1.0.0_10"
part = "140294_f659b875"
name = "assembly.obj"
file_path = os.path.join(datasetPath,part,name)

In [ ]:
mesh = o3d.io.read_triangle_mesh(file_path)
print(f"Loaded mesh: {len(mesh.vertices)} vertices, {len(mesh.triangles)} faces")

TriangleMesh with 111342 points and 122916 triangles.

In [46]:
# 如果没有颜色，给每个顶点一个默认灰色
if not mesh.has_vertex_colors():
    import numpy as np
    mesh.vertex_colors = o3d.utility.Vector3dVector(np.ones((len(mesh.vertices), 3)) * 0.5)


Sampled 300000 points
Downsampled to 300000 points


In [33]:
pcd = mesh.sample_points_uniformly(number_of_points=500000)
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)

N = len(points)
print(f"Sampled {N} points from mesh")

Sampled 500000 points from mesh


In [34]:
# ===== Step 2: 构建 k-NN 邻接图 =====
k = 10  # 每个点连接的邻居数
nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='kd_tree').fit(points)
distances, indices = nbrs.kneighbors(points)

In [35]:
# ===== Step 3: 计算边权重 =====
edges = []
for i in range(N):
    for j in indices[i][1:]:  # 跳过自己
        spatial_dist = np.linalg.norm(points[i] - points[j])
        color_dist = np.linalg.norm(colors[i] - colors[j])
        weight = spatial_dist + 0.5 * color_dist  # 权重 = 空间 + 颜色
        edges.append((i, j, weight))

In [36]:
# ===== Step 4: 简单 Felzenszwalb-like 图合并分割 =====
parent = np.arange(N)

def find(x):
    while parent[x] != x:
        parent[x] = parent[parent[x]]
        x = parent[x]
    return x

def union(a, b):
    ra, rb = find(a), find(b)
    if ra != rb:
        parent[rb] = ra

# 按权重排序
edges.sort(key=lambda x: x[2])

threshold = 0.05  # 分割粒度，越小越细
for i, j, w in edges:
    if w < threshold:
        union(i, j)

In [37]:
# ===== Step 5: 生成分割标签并上色 =====
labels = np.array([find(i) for i in range(N)])
unique_labels = np.unique(labels)
print(f"Found {len(unique_labels)} segments")

# 每个 segment 一个随机颜色
label_to_color = {label: np.random.rand(3) for label in unique_labels}
seg_colors = np.array([label_to_color[l] for l in labels])

# 更新点云颜色
pcd.colors = o3d.utility.Vector3dVector(seg_colors)

Found 496828 segments


In [38]:
o3d.io.write_point_cloud("segmented_pointcloud2.ply", pcd)

True